# User-defined functions: summarizing noise data

This example works with a [model of sound levels for the United States](https://www.nps.gov/subjects/sound/soundmap.htm) produced by the US National Park Service. We want to compute the average noise level for each municipality in Hawaii, but the standard `mean` operation provided by exactextract is not appropriate for the logarithmic [decibel](https://en.wikipedia.org/wiki/Decibel) scale used for sound levels.

To avoid working with large data files, we use the sound level grid provided for Hawaii, along with the [TIGER shapefile](https://www2.census.gov/geo/tiger/TIGER2023/COUSUB/) of 2023 municipal boundaries ("county subdivisions") published by the US Census Bureau:

In [ ]:
sound_fname = 'hawaii/HI_L50dBA_sumDay_exi.tif'
municipalities_fname = 'hawaii/tl_2023_15_cousub.zip'

The sound boundaries are published in UTM 4N (meters), while the municipal boundaries use NAD 1983 geographic coordinates. Rather than pass the filenames to `exact_extract`, we will first load the datasets and reproject the municipal boundaries to the UTM coordinate system used by the sound grid.

In [ ]:
from exactextract import exact_extract
from matplotlib import pyplot
import rasterio
import rasterio.plot
import geopandas as gpd

sound_ds = rasterio.open(sound_fname)
municipalities = gpd.read_file(municipalities_fname).to_crs(sound_ds.crs)

fig, ax = pyplot.subplots()
ax.set_xlim([0.3e6, 1.0e6])
ax.set_ylim([2.0e6, 2.5e6])

rasterio.plot.show(sound_ds, ax=ax)
municipalities.plot(ax=ax, facecolor='none', edgecolor='black', linewidth=0.2)

The following function can be used to compute the mean sound levels:

In [ ]:
import numpy as np

def mean_sound_db(values, coverage):
    # transform values, take average, transform result
    values = np.power(10, values)
    mean = np.average(values, weights=coverage)
    return np.log10(mean)

The user-defined function can now be used in the same way as a built-in summary operation. Unsurprisingly, we see that Honolulu is the loudest municipality in Hawaii.

In [ ]:
results = exact_extract(sound_ds, municipalities, mean_sound_db, include_cols=['GEOID', 'NAME'], output='pandas')
results.sort_values(by=['mean_sound_db'], ascending=False)